In [ ]:
import os
import math
import random
import pandas as pd
import wget

## File paths
atlasFile = 'atlas.csv'
listFile = 'listOfIds.txt'
webFile = 'websites.txt'
resultsFolder = 'results'
library = 'library.txt'

# initializers
ra, dec, radius = 0, 0, 0.0014
id = '0'
stars = []
allCSVs = []
bands = ["I4"]#['I1', 'I2', 'I3', 'I4', 'M1', 'M2', 'M3']
atlasFile = pd.read_csv(atlasFile)
starter = 'https://irsa.ipac.caltech.edu/SIA?COLLECTION=spitzer_sha&RESPONSEFORMAT=CSV&POS=circle'

# Convert Panda series to arrays
objidA = pd.Series(atlasFile['objid']).array
raA = pd.Series(atlasFile['ra']).array
decA = pd.Series(atlasFile['dec']).array

atlasFile

In [ ]:
# takes ids in listOfIds.txt into a list
def listOut():
    list = []
    myfile = open(listFile, "rt")
    contents = myfile.read() + '\n'
    myfile.close()
    while len(contents) > 0:
        index = contents.find('\n')
        id = contents[0:index]
        list.append(id)
        contents = contents[index+1:]
    return(list)

# Asks for Object ID, then returns ra and dec of the object is available
# If not available, display error and ends program
# Edit: replaces spaces in ID with underscores
def search(id):
    for i in range(len(objidA)):
        if id == objidA[i]:
            ra = raA[i]
            dec = decA[i]
            id = id.replace(' ', '_')
            return (id, ra, dec)
    print(f'Error: search()\nInvalid id: {id}')
    exit()

# Rounds input to the 5th decimal and outputs string
def roundCords(num):
    temp = (num*100000)+0.5
    temp = math.floor(temp)
    temp = str(float(temp)/100000)
    return(temp)

# Outputs website
def makeWeb(r, d, rad):
    endP = f'{starter}+{r}+{d}+{rad}'
    return(endP)

# Makes the website and adds to a list
def makeEntry(id):
    id, ra, dec = search(id)
    website = (makeWeb(roundCords(ra),roundCords(dec),roundCords(radius)))
    check = False

    #checks if the entry is already made
    for i in stars:
        if i[0] == id:
            check = True

    if check:
        print(f'{id} entry already in listOfIds.txt file')
    else:
        print(f'{id} entry made!')
        entry = [id, ra, dec, radius, website]
        stars.append(entry)
    return(stars)

# Makes a number of random obj #s
def randomIds(randomN):
    for i in range(randomN):
        websites = makeEntry(objidA[random.randrange(0, len(objidA))])
    return(websites)

# Takes a URL and returns the ending parts
def breakUpUrl(string):
    # Reverse the string to search from the end
    reversed_string = string[::-1]
    # Find the index of the first "\" from the end
    slash_index = reversed_string.find("/")
    # Get the substring after the "\" and reverse it back
    after_slash = reversed_string[:slash_index][::-1]
    # Split the string by "_" and return the result
    parts = after_slash.split("_")
    return parts

# Splits url to look for the wanted bands and if it's a maic.fits, and marks which bands are included
def checkBands(urls):
    rowsToSave = []
    includedBands = []
    linkCounter = 0
    for i in range(len(urls)):
        parts = breakUpUrl(urls[i])
        for band in bands:
            if band == parts[1]:
                if 'maic.fits' == parts[6]:
                    rowsToSave.append(i)
                    linkCounter += 1
                    if band in includedBands:
                        pass
                    else:
                        includedBands.append(band)
    return(rowsToSave, includedBands, linkCounter)

# Only grabs the urls that we are looking for
def trimFat(entry):
    filePath = f'{resultsFolder}/{entry[0]}.csv'
    fat = pd.read_csv(filePath)
    s_ras = pd.Series(fat['s_ra'])
    s_decs = pd.Series(fat['s_dec'])
    obs_ids = pd.Series(fat['obs_id']).array
    urls = pd.Series(fat['access_url']).array
    
    #Searches for what rows to save
    rowsToSave, includedBands, linkCounter = checkBands(urls)
    entry.append(includedBands)
    entry.append(linkCounter)
    
    # Overwrites a CSV file, keeping the 's_ra', 's_dec', 'obs_id', and 'access_url'
    theMeat = []
    for row in rowsToSave:
        oneRow = []
        oneRow.append(obs_ids[row])
        oneRow.append(s_ras[row])
        oneRow.append(s_decs[row])
        oneRow.append(urls[row])
        theMeat.append(oneRow)
    theMeatDf = pd.DataFrame(theMeat, columns=['obs_id', 's_ra', 's_dec', 'access_url'])
    theMeatDf.to_csv(filePath, mode='w')
    return(entry)
    
# Download CSV from provided list from 'stars'
def grabCSV(entry):
    id = entry[0]
    link = entry[4]

    # adds all reference file names in 'results' in a list
    rawResults = []
    for file_path in os.listdir(f"{resultsFolder}"):
        if os.path.isfile(os.path.join(resultsFolder, file_path)):
            rawResults.append(file_path)
    # adds all reference file names in 'results/references' in a list
    rResults = []
    for file_path in os.listdir(f"{resultsFolder}/references"):
        if os.path.isfile(os.path.join(f"{resultsFolder}/references", file_path)):
            if file_path != '.gitignore':
                rResults.append(file_path)

    #checks if the csv is already downloaded
    if (id+'.csv') in rawResults:
        print(f'{id}.csv already in results folder\n')
    elif (id+'.csv') in rResults:
        print(f'{id}.csv already in results/references folder\n')
    else:
        print('Grabbing...')
        wget.download(link, f'{id}.csv')
        os.rename(f'{id}.csv', f'{resultsFolder}/{id}.csv')
        print(f'Downloaded: {id}.csv\n')
        entry = trimFat(entry)
        return(entry)

# Adds the ID, Ra, Dec, how many links, included bands, and reference csv link
def updateLibrary():
    wordsInFile = ''
    print(stars)
    for entry in stars:
        print(entry)
        wordsInFile = wordsInFile + f'ID: {entry[0]}\nRA: {entry[1]} DEC: {entry[2]} RADIUS: {entry[3]}\nNumber of Links: {entry[6]}\nIncluded Bands: '
        for i in range(len(entry[5])):
            wordsInFile = wordsInFile + f'{entry[5][i]} '
        wordsInFile = wordsInFile + f'\n{entry[4]}\n\n'
    f = open(library,'w')
    f.write(wordsInFile)
    f.close
    print('Updated Library!\n')

# Self-explanatory tbh
def downloadFITS(limit):
    fitsFilesFolder = f'{resultsFolder}/fitsFiles'
    # Lists out all CSVs
    referenceCSVs = []
    for file_path in os.listdir(resultsFolder):
        if file_path[-3:] == "csv":
            if os.path.isfile(os.path.join(resultsFolder, file_path)):
                referenceCSVs.append(file_path)
    # Lists out all FITS files in the results/fitsFiles folder
    fitsFilesList = []
    for file_path in os.listdir(fitsFilesFolder):
        if file_path[-4:] == "fits":
            if os.path.isfile(os.path.join(fitsFilesFolder, file_path)):
                referenceCSVs.append(file_path)

    # Starts downloading all CSVs 
    for csv in referenceCSVs:
        # Downloads the FITS files until the limit
        csvDF = pd.read_csv(f'{resultsFolder}/{csv}')
        obsIDs = pd.Series(csvDF['obs_id']).array
        urls = pd.Series(csvDF['access_url'])
        counter = 0
        print(f'Downloading from {csv}...\n')
        while counter < limit and counter < len(urls):
            stringURL = str(urls[counter])
            parts_string = stringURL.split("/")

            parts = breakUpUrl(urls[counter])
            band = parts[1]
            fileName = '_'.join([str(item) for item in parts])

            if (fileName) in fitsFilesList:
                print(f'{fileName} already in results/fitsFiles')
            else:
                wget.download(urls[counter], f'{resultsFolder}/fitsFiles/{fileName}') 
                #({band}){obsIDs[counter]}.fits')
                print(f'\tDownloaded ({fileName}\n')
                counter += 1

        # Moves the CSV file into the results/references folder
        os.rename(f'{resultsFolder}/{csv}', f'{resultsFolder}/references/{csv}')
        print(f'Finished downloading from {csv}\n')
    print(f'Finished downloading FITs files\n')


# Self-explanatory again
def askToDownload():
    ask = input('Download FITS flies?(y/n): ')
    if ask.lower() == 'y':
        number = input('Limit of links per star: ')
        if ask.isdigit:
            downloadFITS(math.floor(int(number)))
        else:
            print('Invalid')
    elif ask.lower != 'n':
        print('Invalid')

# Random links option
def randomLinks():
    number = input('How many stars: ')
    if number.isdigit:
        stars = randomIds(math.floor(int(number)))
        for i in range(len(stars)):
            stars[i] = grabCSV(stars[i])
        print('Downloaded CSV(s)!')
        updateLibrary()
    else:
        print('Invalid')

# NonRandom links option
def nonRandomLinks():
    listOfIds = listOut()
    for obj in range(len(listOfIds)):
        stars = makeEntry(listOfIds[obj])
    for i in range(len(stars)):
        stars[i] = grabCSV(stars[i])
    print('Downloaded CSV(s)!')
    updateLibrary()


In [ ]:
start = input('Look for links?(y/n): ')
if start.lower() == 'y':
    links = input('Random links?(y/n): ')
    if links.lower() == 'y':
        randomLinks()
        askToDownload()
    elif links.lower() == 'n':
        nonRandomLinks()
        askToDownload()
    else:
        print('Invalid')
elif start.lower() == 'n':
    askToDownload()
else:
    print('Invalid')